In [1]:
import pandas as pd

# Load your dataset
df = pd.read_csv("cleaned_flight_data.csv", encoding="ISO-8859-1", low_memory=False)

# Preview the data to understand its structure
print(df.head())

# Check if the flight numbers are reused
flight_counts = df['Flight_Number_Reporting_Airline'].value_counts()
print("Flight number counts:\n", flight_counts)

# Identify the flight numbers that are reused (appear more than once)
reused_flights = flight_counts[flight_counts > 1]
print("\nReused flight numbers:\n", reused_flights)

# Group by flight number and inspect the routes and dates
flight_group = df.groupby('Flight_Number_Reporting_Airline').agg({
    'Origin': 'first',  # Assuming 'Origin' column exists
    'Dest': 'first',    # Assuming 'Dest' column exists
    'FlightDate': lambda x: x.unique()  # Get all unique dates for the flight number
})

# Display the first few reused flight numbers and their details
print("\nFlight numbers with reused occurrences and their routes and dates:\n", flight_group.loc[reused_flights.index])

# Optional: You can also check if flights are codeshared by inspecting other columns like 'Reporting_Airline' or 'Tail_Number'
codeshare_flights = df.groupby('Flight_Number_Reporting_Airline').agg({
    'Reporting_Airline': lambda x: x.unique(),  # Unique reporting airlines for the same flight number
    'Tail_Number': lambda x: x.unique()  # Unique aircraft tail numbers
})
print("\nCodeshare flight details:\n", codeshare_flights)

   Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate Reporting_Airline  \
0  1998        1      1           2          5  1998-01-02                NW   
1  2009        2      5          28          4  2009-05-28                FL   
2  2013        2      6          29          6  2013-06-29                MQ   
3  2010        3      8          31          2  2010-08-31                DL   
4  2006        1      1          15          7  2006-01-15                US   

   DOT_ID_Reporting_Airline IATA_CODE_Reporting_Airline Tail_Number  ...  \
0                     19386                          NW      N297US  ...   
1                     20437                          FL      N946AT  ...   
2                     20398                          MQ      N665MQ  ...   
3                     19790                          DL      N6705Y  ...   
4                     20355                          US      N504AU  ...   

   Div1WheelsOff  Div1TailNum  Div2Airport  Div2AirportID Div2

In [2]:
# Create binary target variable 'DelayCategory': 1 if there was an arrival delay (ArrDelay > 0), 0 if there wasn't
df['DelayCategory'] = df.apply(lambda row: 1 if row['ArrDelay'] > 0 else 0, axis=1)

# Group by Flight Number and sum the DelayCategory to count delays
delays_by_flight = df.groupby('Flight_Number_Reporting_Airline')['DelayCategory'].sum().reset_index()

# Sort by the number of delays in descending order
delays_by_flight_sorted = delays_by_flight.sort_values(by='DelayCategory', ascending=False)

# Show the flight numbers with the most delays
print(delays_by_flight_sorted.head())  # You can adjust the number of rows to display as needed

     Flight_Number_Reporting_Airline  DelayCategory
504                              505            634
418                              419            621
439                              440            609
64                                65            605
507                              508            603
